<a href="https://colab.research.google.com/github/Remyar12/news_summary/blob/main/news_summarization_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers  rouge-score

  Preparing metadata (setup.py) ... done


In [ ]:
!pip install rouge-score

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

import pandas as pd
import transformers
from torch.utils.data import Dataset, DataLoader


from transformers import AutoTokenizer, AutoModel
sentenc_model_name = "sentence-transformers/paraphrase-MiniLM-L3-v2"
tokenizer = AutoTokenizer.from_pretrained(sentenc_model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

import os

sum_dir = "drive/MyDrive/mlexperiments/extractivesummarization/" # location to store and load models

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Define the path to the CSV file
csv_file_path = "/content/test.csv"

# Load the dataset into a pandas DataFrame
try:
    df = pd.read_csv(csv_file_path)
    print("CSV file loaded successfully.")
except FileNotFoundError:
    print(f"File not found: {csv_file_path}")
    df = pd.DataFrame()  # Create an empty DataFrame if the file is not found

# Display the shape of the dataset
print("Original dataset shape:", df.shape)

# Check the contents of the dataset
print(df.head())

# Split the dataset into training and testing sets
if not df.empty:
    train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

    # Display the shapes of the training and testing sets
    print("Training set shape:", train_df.shape)
    print("Testing set shape:", test_df.shape)
else:
    print("The dataset is empty or not loaded correctly.")


CSV file loaded successfully.
Original dataset shape: (11490, 3)
                                         id  \
0  92c514c913c0bdfe25341af9fd72b29db544099b   
1  2003841c7dc0e7c5b1a248f9cd536d727f27a45a   
2  91b7d2311527f5c2b63a65ca98d21d9c92485149   
3  caabf9cbdf96eb1410295a673e953d304391bfbb   
4  3da746a7d9afcaa659088c8366ef6347fe6b53ea   

                                             article  \
0  Ever noticed how plane seats appear to be gett...   
1  A drunk teenage boy had to be rescued by secur...   
2  Dougie Freedman is on the verge of agreeing a ...   
3  Liverpool target Neto is also wanted by PSG an...   
4  Bruce Jenner will break his silence in a two-h...   

                                          highlights  
0  Experts question if  packed out planes are put...  
1  Drunk teenage boy climbed into lion enclosure ...  
2  Nottingham Forest are close to extending Dougi...  
3  Fiorentina goalkeeper Neto has been linked wit...  
4  Tell-all interview with the reality T

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel, AdamW
from torch.cuda.amp import GradScaler, autocast
from tqdm import tqdm
import time
import cProfile
import torch.nn as nn

class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len_input):
        self.data = dataframe
        self.articles = self.data['article']
        self.highlights = self.data['highlights']
        self.tokenizer = tokenizer
        self.max_len_input = max_len_input

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        article = self.articles[idx]
        highlights = self.highlights[idx]

        input_encoding = self.tokenizer.encode_plus(
            article,
            add_special_tokens=True,
            max_length=self.max_len_input,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )

        return {
            'input_ids': input_encoding['input_ids'].flatten(),
            'attention_mask': input_encoding['attention_mask'].flatten(),
            'highlights': highlights
        }

# Corrected file paths
train_file = "/content/output.csv.txttrain_bdf.csv"
test_file = "/content/output.csv.txttest_bdf.csv"

train_df = pd.read_csv(train_file)
test_df = pd.read_csv(test_file)

# Reset indices
train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

# Constants
TRAIN_BATCH_SIZE = 4  # Reduce batch size to avoid memory issues
EPOCHS = 3
LEARNING_RATE = 2e-5
MAX_LEN_INPUT = 256
MODEL_NAME = 'sentence-transformers/paraphrase-MiniLM-L3-v2'
GRADIENT_ACCUMULATION_STEPS = 4

# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Define CustomDataset and DataLoader
train_dataset = CustomDataset(train_df, tokenizer, MAX_LEN_INPUT)
train_loader = DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)

# Verify DataLoader
data_iter = iter(train_loader)
batch = next(data_iter)
print(batch)

# Define the model
model = AutoModel.from_pretrained(MODEL_NAME)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

# Define optimizer and loss function
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
scaler = GradScaler()
criterion = nn.MSELoss()  # Using MSELoss as a placeholder

# Define train function
def train(epoch):
    model.train()
    tr_loss = 0
    nb_tr_steps = 0
    start_time = time.time()

    for step, data in enumerate(tqdm(train_loader, 0)):
        input_ids = data['input_ids'].to(device, dtype=torch.long)
        attention_mask = data['attention_mask'].to(device, dtype=torch.long)
        highlights = data['highlights']

        optimizer.zero_grad()

        with autocast():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            # Get the sentence embeddings
            sentence_embeddings = outputs.last_hidden_state.mean(dim=1)

            # Dummy targets for demonstration purposes
            dummy_targets = torch.randn_like(sentence_embeddings).to(device)
            loss = criterion(sentence_embeddings, dummy_targets)

        scaler.scale(loss).backward()

        if (step + 1) % GRADIENT_ACCUMULATION_STEPS == 0:
            scaler.step(optimizer)
            scaler.update()

        tr_loss += loss.item() * GRADIENT_ACCUMULATION_STEPS
        nb_tr_steps += 1

        if step % 100 == 0:
            elapsed_time = time.time() - start_time
            print(f"Step {step}, Loss: {tr_loss / nb_tr_steps}, Elapsed Time: {elapsed_time:.2f}s")
            start_time = time.time()

    print(f'Epoch {epoch}, Loss: {tr_loss / nb_tr_steps}')

# Profile the training function
cProfile.run('train(0)', sort='cumtime')

# Train the model
for epoch in range(EPOCHS):
    train(epoch)


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


{'input_ids': tensor([[ 101, 1037, 2624,  ..., 2001, 3013,  102],
        [ 101, 2005, 1996,  ..., 2031, 2025,  102],
        [ 101, 2074, 2420,  ..., 1012, 1045,  102],
        [ 101, 9433, 2333,  ..., 1012, 2864,  102]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]]), 'highlights': ['Manuel Garza Jr. died from a lethal injection on Thursday .\nHe was put to death for killing a San Antonio police officer in 2001 .\nHe shot officer John Riojas after a struggle that ended with Riojas being shot in the head .\nGarza was pronounced dead 26 minutes after being injected .', "For the first time in 75 years the California Department of Water Resources has found no snow during its survey of the Sierra Nevada in early April .\nNASA images showing  the snowpack across the Tuolumne River Basin   show a marked decrease since last April .\nCalifornia depends on the snow to melt into rivers a

model.safetensors:   0%|          | 0.00/69.6M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 0/2298 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
  0%|          | 4/2298 [00:01<13:02,  2.93it/s]

Step 0, Loss: 3.85542368888855, Elapsed Time: 1.54s


...

Step 2100, Loss: 4.006049758419997, Elapsed Time: 3.19s


 96%|█████████▌| 2203/2298 [01:21<00:05, 18.46it/s]

Step 2200, Loss: 4.005665531054458, Elapsed Time: 4.89s


100%|██████████| 2298/2298 [01:25<00:00, 27.03it/s]

Epoch 2, Loss: 4.005854491465397


In [ ]:
import os
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from rouge_score import rouge_scorer

MODEL_NAME = 'sentence-transformers/paraphrase-MiniLM-L3-v2'

class CustomTextDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length):
        self.data = dataframe.reset_index(drop=True)  # Reset indices
        self.articles = self.data['article']
        self.highlights = self.data['highlights']
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        article = str(self.articles[idx])
        highlight = str(self.highlights[idx])

        article_encoding = self.tokenizer.encode_plus(
            article,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        highlight_encoding = self.tokenizer.encode_plus(
            highlight,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        article_input_ids = article_encoding['input_ids'].flatten()
        article_attention_mask = article_encoding['attention_mask'].flatten()
        highlight_input_ids = highlight_encoding['input_ids'].flatten()
        highlight_attention_mask = highlight_encoding['attention_mask'].flatten()

        return {
            'article_input_ids': article_input_ids,
            'article_attention_mask': article_attention_mask,
            'highlight_input_ids': highlight_input_ids,
            'highlight_attention_mask': highlight_attention_mask,
            'highlight': highlight  # Keep original highlight for evaluation
        }

def validate_model(model, dataloader, tokenizer, device):
    model.eval()
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    all_scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}
    cosine_similarities = []

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Validating"):
            article_input_ids = batch['article_input_ids'].to(device)
            article_attention_mask = batch['article_attention_mask'].to(device)
            highlight_input_ids = batch['highlight_input_ids'].to(device)
            highlight_attention_mask = batch['highlight_attention_mask'].to(device)
            highlights = batch['highlight']

            # Get the embeddings for articles and highlights
            article_embeddings = model(input_ids=article_input_ids, attention_mask=article_attention_mask).last_hidden_state.mean(dim=1)
            highlight_embeddings = model(input_ids=highlight_input_ids, attention_mask=highlight_attention_mask).last_hidden_state.mean(dim=1)

            # Compute cosine similarity
            for article_emb, highlight_emb in zip(article_embeddings, highlight_embeddings):
                cos_sim = cosine_similarity(article_emb.cpu().numpy().reshape(1, -1), highlight_emb.cpu().numpy().reshape(1, -1))[0][0]
                cosine_similarities.append(cos_sim)

            # Decode the articles and highlights
            articles = [tokenizer.decode(ids, skip_special_tokens=True) for ids in article_input_ids]
            reference_summaries = [highlight for highlight in highlights]

            for article, ref_sum in zip(articles, reference_summaries):
                scores = scorer.score(ref_sum, article)
                for key in scores:
                    all_scores[key].append(scores[key].fmeasure)

    # Calculate average ROUGE scores
    avg_rouge_scores = {key: sum(scores) / len(scores) for key, scores in all_scores.items()}
    avg_cosine_similarity = sum(cosine_similarities) / len(cosine_similarities)

    return avg_rouge_scores, avg_cosine_similarity

# Load your test dataset
test_df = pd.read_csv("/content/output.csv.txttest_bdf.csv")

# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Use a smaller subset for quick testing
test_subset_df = test_df.sample(n=100, random_state=42)

# Debugging: Print the first few rows of the subset
print(test_subset_df.head())

# Initialize your dataset
test_dataset = CustomTextDataset(test_subset_df, tokenizer, max_length=128)  # Reduced lengths

# Debugging: Check the length of the dataset
print("Length of test_dataset:", len(test_dataset))

# Create DataLoader for the test dataset
testing_loader = DataLoader(test_dataset, batch_size=4, shuffle=False, num_workers=4)  # Adjusted parameters

# Debugging: Check the first batch
for batch in testing_loader:
    print(batch)
    break

# Load pre-trained model
model = AutoModel.from_pretrained(MODEL_NAME)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Validate the model using the testing_loader
rouge_scores, avg_cosine_similarity = validate_model(model, testing_loader, tokenizer, device)
print("ROUGE scores on test data:", rouge_scores)
print("Average Cosine Similarity on test data:", avg_cosine_similarity)


                                            id  \
1085  b1585b611a9007d7b86851b6a5c9d32a47040aa8   
2296  f7d9f33ac38ce839c81c7188d33c4e09ce11094e   
208   da39f68c5d9a413e0759fc6bff8260f76ed1ac59   
1178  84bdd724457a4c2161844fb4ff91fefa5465e9aa   
282   b3a62e1415b13dfe0938e206d6353bc57dbd354a   

                                                article  \
1085  It's truly squeaky bum time in the Premier Lea...   
2296  Forget fetching sticks. This musically-minded ...   
208   Martin 'Mad Dog' Allen returns with his latest...   
1178  Jeremy Scott’s spring/summer 2015 collection f...   
282   Joe beats Sachin to 2,000 . Joe Root became th...   

                                             highlights  
1085  Just nine points separates the bottom seven cl...  
2296  Sadie the German Shepherd was filmed at home i...  
208   Martin Allen believes it's a crucial time for ...  
1178  Olivia Phillips, 30, rocked head-to-toe Moschi...  
282   Ben Stokes' dad Gerard tweets his approval of ..

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


{'article_input_ids': tensor([[  101,  2009,  1005,  1055,  5621, 29552,  2100, 26352,  2051,  1999,
          1996,  4239,  2223,  9591,  2645,  2004,  2074,  3157,  2685, 18600,
          1996,  3953,  2698,  2780,  1012,  2998, 21397,  5176,  2070,  1997,
          1996, 10489,  1999,  1998,  2426,  1996,  3899,  2954,  2054,  2027,
          2514,  2003,  3223,  2005,  2068,  2000,  4468,  1996,  4530,  2023,
          2161,  1012,  3160,  1024,  1005,  2054,  2097,  2009,  2202,  2005,
          2115,  2252,  2000,  2994,  1999,  1996,  4239,  2223,  1029,  1005,
          3782, 15092,  1006,  1053, 18098,  1007,  1005,  2065,  2057,  2663,
          2093,  2399,  1045,  2228,  2057,  2097,  2994,  2039,  1012,  2009,
          2453,  2202,  2625,  1012,  1045,  1005,  1049,  2025,  3038,  2008,
          1005,  1055,  1996, 15764,  3815,  1011,  2021,  2057,  2428,  2342,
          2000,  2707,  3045,  1010,  3225,  2007,  2023,  5353,  1012,  2000,
          2079,  2008,  1010, 

Validating: 100%|██████████| 25/25 [00:01<00:00, 18.49it/s]

ROUGE scores on test data: {'rouge1': 0.3967823638710646, 'rouge2': 0.1852990763553967, 'rougeL': 0.2550684468159702}
Average Cosine Similarity on test data: 0.4350149495899677


In [1]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModel

model = AutoModel.from_pretrained("sentence-transformers/paraphrase-MiniLM-L3-v2")

# Define training function (example)
def train_model(model, train_loader):
    # Training loop here
    pass

# Train your model (example)
# train_model(model, train_loader)

# Save the model
model_save_path = "/content/minilm_bal_exsum.pth"
torch.save(model.state_dict(), model_save_path)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/69.6M [00:00<?, ?B/s]

In [2]:
! pip install streamlit transformers torch newspaper3k sumy



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 43.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 86.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 11.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6

In [3]:
!pip install streamlit


In [4]:
!pip install sumy


In [5]:
!pip install newspaper3k


In [6]:
!pip install transformers torch


In [7]:
!npm install -g localtunnel


/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.2
added 22 packages from 22 contributors in 3.704s


In [8]:
! pip install streamlit -q

In [9]:
!pip install nltk

In [10]:
!pip install streamlit sentence-transformers newspaper3k nltk scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 4.3 MB/s eta 0:00:00


In [11]:
!wget -q -O - ipv4.icanhazip.com

34.143.156.106


In [14]:
%%writefile app.py
import streamlit as st
from sentence_transformers import SentenceTransformer, util
import newspaper
import nltk
nltk.download('punkt')


model = SentenceTransformer('sentence-transformers/paraphrase-MiniLM-L3-v2')

def summarize_news(text, num_sentences=2):
    sentences = nltk.sent_tokenize(text)
    embeddings = model.encode(sentences, convert_to_tensor=True)
    cosine_similarities = util.pytorch_cos_sim(embeddings, embeddings)
    sentence_indices = cosine_similarities.sum(axis=1).topk(num_sentences).indices
    representative_sentences = [sentences[i] for i in sentence_indices]
    summarized_news = " ".join(representative_sentences)
    return summarized_news

def scrape_and_summarize(url, num_sentences=2):
    article = newspaper.Article(url)
    article.download()
    article.parse()
    text = article.text
    summarized_news = summarize_news(text, num_sentences)
    return summarized_news

def main():
    st.title("NEWS SUMMARIZATION APP")
    choice = st.radio("Choose Input Type", ("Enter News", "Enter URL"))
    num_sentences = st.slider("Number of Sentences in Summary:", min_value=1, max_value=3, value=2)

    if choice == "Enter News":
        text = st.text_area("Enter News:")
        if st.button("Summarize News"):
            if text:
                summarized_news = summarize_news(text, num_sentences)
                st.subheader("Summary:")
                st.write(summarized_news)
            else:
                st.warning("Please enter the news to summarize.")

    elif choice == "Enter URL":
        url = st.text_input("Enter URL of the News Article:")
        if st.button("Summarize URL"):
            if url:
                summarized_news = scrape_and_summarize(url, num_sentences)
                st.subheader("Summary:")
                st.write(summarized_news)
            else:
                st.warning("Please enter the URL of the news article.")

if __name__ == "__main__":
    main()


Overwriting app.py


In [15]:
!streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.143.156.106:8501

npx: installed 22 in 2.577s
your url is: https://six-foxes-stare.loca.lt
2024-06-25 05:18:23.357519: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-25 05:18:23.357570: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-25 05:18:23.467801: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-25 05:18:23.690021: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimize